In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout

In [ ]:
# Function to load and preprocess images
def load_images_and_masks(image_path, mask_path):
    image_filenames = os.listdir(image_path)
    images = [cv2.resize(cv2.imread(os.path.join(image_path, filename), cv2.IMREAD_GRAYSCALE), (224, 224)) for filename in image_filenames]
    mask_filenames = os.listdir(mask_path)
    masks = [cv2.resize(cv2.imread(os.path.join(mask_path, filename), cv2.IMREAD_GRAYSCALE), (224, 224)) for filename in mask_filenames]
    return np.array(images) / 255.0, np.array(masks) / 255.0

In [ ]:
covid19_img = '/content/drive/MyDrive/CovidXRayImages/COVID-19 /images'
covid19_lm = '/content/drive/MyDrive/CovidXRayImages/COVID-19 /lung masks'
noncovid_img = '/content/drive/MyDrive/CovidXRayImages/Non-COVID/images'
noncovid_lm = '/content/drive/MyDrive/CovidXRayImages/Non-COVID/lung masks'
normal_img = '/content/drive/MyDrive/CovidXRayImages/Normal/images'
normal_lm ='/content/drive/MyDrive/CovidXRayImages/Normal/lung masks'

In [ ]:
# Load images and masks
covid19_images, covid19_masks = load_images_and_masks(covid19_img, covid19_lm)
noncovid_images, noncovid_masks = load_images_and_masks(noncovid_img, noncovid_lm)
normal_images, normal_masks = load_images_and_masks(normal_img, normal_lm)

In [ ]:
# Combine images and masks
all_images = np.concatenate((covid19_images, noncovid_images, normal_images), axis=0)
all_masks = np.concatenate((covid19_masks, noncovid_masks, normal_masks), axis=0)

In [ ]:
# Assign labels: 0 for Covid-19, 1 for Non-Covid, 2 for Normal
labels = np.concatenate((np.zeros(len(covid19_images)), np.ones(len(noncovid_images)), 2*np.ones(len(normal_images))))


In [ ]:
# Shuffle the data while keeping labels intact
indices = np.arange(len(all_images))
np.random.shuffle(indices)
all_images = all_images[indices]
all_masks = all_masks[indices]
labels = labels[indices]


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, labels, test_size=0.2, random_state=42)

In [ ]:
len(y_train)

4333

In [ ]:
# Define image dimensions
image_height, image_width = X_train.shape[1], X_train.shape[2]

In [ ]:
image_height

224

In [ ]:
image_width

224

In [ ]:
# Expand dimensions to add a channel dimension
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Create an ImageDataGenerator for data augmentation
#datagen = ImageDataGenerator(
 #   rotation_range=10,
  #  width_shift_range=0.1,
   # height_shift_range=0.1,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #vertical_flip=True,
    #fill_mode='nearest'
#)


In [ ]:
# Fit the ImageDataGenerator on the training data
#datagen.fit(X_train)

In [ ]:
 #Define early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Build CNN model for X-ray images
input_img = Input(shape=(image_height, image_width, 1))
x = Conv2D(16, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.3)(x)  # Adding dropout
xray_output = Dense(32, activation='relu')(x)

In [ ]:
# Build CNN model for lung mask images
input_mask = Input(shape=(image_height, image_width, 1))
y = Conv2D(16, (3, 3), activation='relu')(input_mask)
y = MaxPooling2D((2, 2))(y)
y = Conv2D(32, (3, 3), activation='relu')(y)
y = MaxPooling2D((2, 2))(y)
y = Conv2D(64, (3, 3), activation='relu')(y)
y = MaxPooling2D((2, 2))(y)
y = Flatten()(y)
y = Dropout(0.3)(y)  # Adding dropout
mask_output = Dense(32, activation='relu')(y)

In [ ]:
combined = Concatenate()([xray_output, mask_output])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.3)(z)  # Adding dropout
output = Dense(3, activation='softmax')(z)  # Output layer with 3 classes


In [ ]:
# Create model
model = Model(inputs=[input_img, input_mask], outputs=output)

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Early Stopping: Implement early stopping during training to monitor the validation loss and stop training when it starts to increase, indicating overfitting.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# Train model with augmented data
#model.fit(
  #  datagen.flow([X_train, all_masks[:len(X_train)]], y_train, batch_size=32),
   # epochs=30,
   # steps_per_epoch=len(X_train) / 32,
   # validation_data=([X_test, all_masks[len(X_train):]], y_test),
   # callbacks=[early_stopping]
#)

In [ ]:
# Train model
model.fit([X_train, all_masks[:len(X_train)]], y_train, epochs=30, batch_size=32, validation_data=([X_test, all_masks[len(X_train):]], y_test), callbacks=[early_stopping])


Epoch 1/30
136/136 [==============================] - 12s 31ms/step - loss: 0.8779 - accuracy: 0.6047 - val_loss: 0.6484 - val_accuracy: 0.7472
Epoch 2/30
136/136 [==============================] - 2s 18ms/step - loss: 0.6467 - accuracy: 0.7480 - val_loss: 0.5671 - val_accuracy: 0.8017
Epoch 3/30
136/136 [==============================] - 2s 18ms/step - loss: 0.5366 - accuracy: 0.7994 - val_loss: 0.5240 - val_accuracy: 0.7989
Epoch 4/30
136/136 [==============================] - 2s 17ms/step - loss: 0.4690 - accuracy: 0.8195 - val_loss: 0.4989 - val_accuracy: 0.8192
Epoch 5/30
136/136 [==============================] - 2s 17ms/step - loss: 0.3762 - accuracy: 0.8569 - val_loss: 0.5353 - val_accuracy: 0.8100
Epoch 6/30
136/136 [==============================] - 2s 17ms/step - loss: 0.2968 - accuracy: 0.8867 - val_loss: 0.5287 - val_accuracy: 0.8100
Epoch 7/30
136/136 [==============================] - 2s 17ms/step - loss: 0.2473 - accuracy: 0.9075 - val_loss: 0.5704 - val_accuracy: 0.791

In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate([X_test, all_masks[len(X_train):]], y_test)
print("Test Accuracy:", test_acc)

34/34 [==============================] - 0s 7ms/step - loss: 0.4989 - accuracy: 0.8192
Test Accuracy: 0.8191881775856018


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 16)         160       ['input_1[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 222, 222, 16)         160       ['input_2[0][0]']             
                                                                                              

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Make predictions on test data
y_pred = model.predict([X_test, all_masks[len(X_train):]])
y_pred_classes = np.argmax(y_pred, axis=1)

34/34 [==============================] - 0s 6ms/step


In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       375
         1.0       0.82      0.76      0.79       379
         2.0       0.81      0.81      0.81       330

    accuracy                           0.82      1084
   macro avg       0.82      0.82      0.82      1084
weighted avg       0.82      0.82      0.82      1084



In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[333  26  16]
 [ 44 288  47]
 [ 26  37 267]]


In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# Initialize list to store test accuracies
test_accuracies = []


In [ ]:
# Iterate over folds
for fold, (train_index, test_index) in enumerate(skf.split(all_images, labels), 1):
    print(f'Fold {fold}')

    # Split data into training and testing sets
    X_train, X_test = all_images[train_index], all_images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Define CNN model
    input_img = Input(shape=(224, 224, 1))
    x = Conv2D(16, (3, 3), activation='relu')(input_img)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)
    model = Model(inputs=input_img, outputs=output)

    # Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("Test Accuracy:", test_acc)

    # Append test accuracy to list
    test_accuracies.append(test_acc)

Fold 1
Epoch 1/10
113/113 [==============================] - 4s 25ms/step - loss: 0.9751 - accuracy: 0.5353 - val_loss: 0.8120 - val_accuracy: 0.6412
Epoch 2/10
113/113 [==============================] - 1s 12ms/step - loss: 0.7409 - accuracy: 0.6893 - val_loss: 0.7211 - val_accuracy: 0.7027
Epoch 3/10
113/113 [==============================] - 1s 12ms/step - loss: 0.6358 - accuracy: 0.7427 - val_loss: 0.6382 - val_accuracy: 0.7436
Epoch 4/10
113/113 [==============================] - 1s 12ms/step - loss: 0.5925 - accuracy: 0.7693 - val_loss: 0.6188 - val_accuracy: 0.7691
Epoch 5/10
113/113 [==============================] - 1s 12ms/step - loss: 0.5415 - accuracy: 0.7951 - val_loss: 0.6463 - val_accuracy: 0.7447
Epoch 6/10
113/113 [==============================] - 1s 12ms/step - loss: 0.5097 - accuracy: 0.8048 - val_loss: 0.6069 - val_accuracy: 0.7674
Epoch 7/10
113/113 [==============================] - 1s 12ms/step - loss: 0.4694 - accuracy: 0.8247 - val_loss: 0.6131 - val_accuracy:

In [ ]:

# Calculate and print average test accuracy
avg_test_accuracy = np.mean(test_accuracies)
print("Average Test Accuracy:", avg_test_accuracy)

Average Test Accuracy: 0.8076414068539938


Emperical tuning

In [ ]:
def create_cnn_model(image_height, image_width, dropout_rate, num_filters, kernel_size, pool_size, dense_units):
    input_img = Input(shape=(image_height, image_width, 1))
    x = Conv2D(num_filters, kernel_size, activation='relu')(input_img)
    x = MaxPooling2D(pool_size)(x)
    x = Conv2D(num_filters * 2, kernel_size, activation='relu')(x)
    x = MaxPooling2D(pool_size)(x)
    x = Conv2D(num_filters * 4, kernel_size, activation='relu')(x)  # New convolutional layer
    x = MaxPooling2D(pool_size)(x)                                  # New pooling layer
    x = Flatten()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)
    model = Model(inputs=input_img, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Define image dimensions
image_height, image_width = X_train.shape[1], X_train.shape[2]


In [ ]:
# Define hyperparameters for tuning
hyperparameters = [
    {'dropout_rate': 0.6, 'num_filters': 16, 'kernel_size': (3, 3), 'pool_size': (2, 2), 'dense_units': 32},
    {'dropout_rate': 0.5, 'num_filters': 32, 'kernel_size': (3, 3), 'pool_size': (2, 2), 'dense_units': 64},
    {'dropout_rate': 0.4, 'num_filters': 64, 'kernel_size': (3, 3), 'pool_size': (2, 2), 'dense_units': 128}
]


In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Perform empirical tuning
for i, params in enumerate(hyperparameters, 1):
    print(f"Round {i} Hyperparameters:", params)
    model = create_cnn_model(X_train.shape[1], X_train.shape[2], **params)
    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping]
    )
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("Test Accuracy:", test_acc)

Round 1 Hyperparameters: {'dropout_rate': 0.6, 'num_filters': 16, 'kernel_size': (3, 3), 'pool_size': (2, 2), 'dense_units': 32}
Epoch 1/30
113/113 [==============================] - 4s 22ms/step - loss: 0.8620 - accuracy: 0.6146 - val_loss: 0.7856 - val_accuracy: 0.6554
Epoch 2/30
113/113 [==============================] - 1s 12ms/step - loss: 0.6452 - accuracy: 0.7417 - val_loss: 0.6738 - val_accuracy: 0.7252
Epoch 3/30
113/113 [==============================] - 1s 12ms/step - loss: 0.5682 - accuracy: 0.7810 - val_loss: 0.6294 - val_accuracy: 0.7512
Epoch 4/30
113/113 [==============================] - 1s 12ms/step - loss: 0.5066 - accuracy: 0.8117 - val_loss: 0.5912 - val_accuracy: 0.7557
Epoch 5/30
113/113 [==============================] - 1s 12ms/step - loss: 0.4351 - accuracy: 0.8331 - val_loss: 0.5835 - val_accuracy: 0.7690
Epoch 6/30
113/113 [==============================] - 1s 12ms/step - loss: 0.3738 - accuracy: 0.8533 - val_loss: 0.5453 - val_accuracy: 0.7861
Epoch 7/30
11

In [ ]:
#performing auc on the model
from sklearn.metrics import roc_auc_score
y_pred_probs = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred_probs, multi_class='ovr')
print("AUC:", auc)

57/57 [==============================] - 1s 10ms/step
AUC: 0.9303321473610682


In [ ]:
#performing confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

57/57 [==============================] - 1s 10ms/step
Confusion Matrix:
[[574  36  24]
 [ 50 438 112]
 [ 59  75 437]]
